In [ ]:
!pip install torchdata

In [ ]:
import torch
import torch.nn as nn

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import torchtext.transforms as T
from torch.hub import load_state_dict_from_url

padding_idx = 1
bos_idx = 0
eos_idx = 2
max_seq_len = 256
vocab_path = r"https://download.pytorch.org/models/text/xlmr.vocab.pt"
spm_model_path = r"https://download.pytorch.org/models/text/xlmr.sentencepiece.bpe.model"
#spm_model_path = r"https://huggingface.co/IDEA-CCNL/Erlangshen-DeBERTa-v2-186M-Chinese-SentencePiece/resolve/main/spm.model"

text_transform = T.Sequential(
    T.SentencePieceTokenizer(spm_model_path),
    T.VocabTransform(load_state_dict_from_url(vocab_path)),
    T.Truncate(max_seq_len - 2),
    T.AddToken(token=bos_idx, begin=True),
    T.AddToken(token=eos_idx, begin=False),
)

from torchdata.datapipes.iter import IterableWrapper
train_datapipe = IterableWrapper(["/kaggle/input/intent-recog/train.csv"])
train_datapipe = train_datapipe.open_files(encoding="utf-8").parse_csv()
train_datapipe = train_datapipe.shuffle().sharding_filter()

In [ ]:
test_datapipe = IterableWrapper(["/kaggle/input/intent-recog/test.csv"])
test_datapipe = test_datapipe.open_files(encoding="utf-8").parse_csv()
test_datapipe = test_datapipe.shuffle().sharding_filter()

In [ ]:
batch_size = 128

def batch_transform(x):
    return text_transform(x[0]), int(x[1])-1

train_datapipe = train_datapipe.map(batch_transform)
train_datapipe = train_datapipe.batch(batch_size)
train_datapipe = train_datapipe.rows2columnar(["token_ids", "target"])


from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_datapipe, batch_size=None)

In [ ]:
test_datapipe = test_datapipe.map(batch_transform)
test_datapipe = test_datapipe.batch(batch_size)
test_datapipe = test_datapipe.rows2columnar(["token_ids", "target"])
test_dataloader = DataLoader(test_datapipe, batch_size=None)

In [ ]:
num_classes = 14
input_dim = 768

from torchtext.models import RobertaClassificationHead, XLMR_BASE_ENCODER

classifier_head = RobertaClassificationHead(num_classes=num_classes, input_dim=input_dim)
model = XLMR_BASE_ENCODER.get_model(head=classifier_head)
model.to(DEVICE)

In [ ]:
import torchtext.functional as F
from torch.optim import AdamW

learning_rate = 1e-5
optim = AdamW(model.parameters(), lr=learning_rate)
criteria = nn.CrossEntropyLoss()


def train_step(input, target):
    output = model(input)
    loss = criteria(output, target)
    optim.zero_grad()
    loss.backward()
    optim.step()


def eval_step(input, target):
    output = model(input)
    loss = criteria(output, target).item()
    acc = (output.argmax(1) == target).type(torch.float).mean()
    print(acc)
    return float(loss), acc

def evaluate():
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
    counter = 0
    with torch.no_grad():
        for batch in test_dataloader:
            input = F.to_tensor(batch["token_ids"], padding_value=padding_idx).to(DEVICE)
            target = torch.tensor(batch["target"]).to(DEVICE)
            loss, predictions = eval_step(input, target)
            total_loss += loss
            correct_predictions += predictions
            
            total_predictions += 1
            counter += 1

    return total_loss / counter, correct_predictions / total_predictions


In [ ]:
num_epochs = 20

for e in range(num_epochs):
    for batch in train_dataloader:
        input = F.to_tensor(batch["token_ids"], padding_value=padding_idx).to(DEVICE)
        target = torch.tensor(batch["target"]).to(DEVICE)
        train_step(input, target)
    loss, accuracy = evaluate()
    print("TEST Epoch = [{}], loss = [{}], accuracy = [{}]".format(e, loss, accuracy))
torch.save(model, "./model.pt")

In [ ]:
!ls